In [49]:
import sqlite3
import pandas as pd
from io import StringIO

# Tạo database trong RAM
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

print("Đang tạo các bảng...")

# Tạo bảng với JSON support (SQLite 3.38+)
cur.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    row_key TEXT PRIMARY KEY,
    value TEXT
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS objects (
    row_key TEXT PRIMARY KEY,
    value TEXT
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS access_policy (
    row_key TEXT PRIMARY KEY,
    value TEXT
);
""")

print("✓ Đã tạo các bảng thành công")

# 3. Xóa dữ liệu cũ
cur.execute("DELETE FROM subjects;")
cur.execute("DELETE FROM objects;")
cur.execute("DELETE FROM access_policy;")

# 4. Insert dữ liệu
print("\nĐang insert dữ liệu...")

import json

# Subjects
subjects_data = [
    ('user:1', json.dumps({"username":"alice","group_name":"dev","department":"it"})),
    ('user:2', json.dumps({"username":"bob","group_name":"dev","department":"it"})),
    ('user:3', json.dumps({"username":"charlie","group_name":"qa","department":"it"})),
    ('user:4', json.dumps({"username":"david","group_name":"recruiting","department":"hr"})),
    ('user:5', json.dumps({"username":"eve","group_name":"payroll","department":"hr"})),
    ('user:6', json.dumps({"username":"frank","group_name":"north","department":"sales"})),
    ('user:7', json.dumps({"username":"grace","group_name":"south","department":"sales"}))
]

cur.executemany("INSERT INTO subjects (row_key, value) VALUES (?, ?);", subjects_data)

# Objects
objects_data = [
    ('file:1', json.dumps({"company":"ABC","branch":"engineering","file_name":"product_spec.pdf"})),
    ('file:2', json.dumps({"company":"ABC","branch":"engineering","file_name":"architecture.doc"})),
    ('file:3', json.dumps({"company":"ABC","branch":"research","file_name":"research_paper.pdf"})),
    ('file:4', json.dumps({"company":"financecorp","branch":"main","file_name":"financial_report.xlsx"})),
    ('file:5', json.dumps({"company":"financecorp","branch":"audit","file_name":"audit_log.txt"}))
]

cur.executemany("INSERT INTO objects (row_key, value) VALUES (?, ?);", objects_data)

# Access Policy
policy_data = [
    ('ap:1', json.dumps({"username":"alice","company":"ABC","branch":"engineering","file_name":"product_spec.pdf","allowed":True})),
    ('ap:2', json.dumps({"username":"alice","company":"ABC","branch":"engineering","file_name":"architecture.doc","allowed":True})),
    ('ap:3', json.dumps({"username":"bob","group_name":"dev","department":"it","company":"ABC","branch":"engineering","file_name":"product_spec.pdf","allowed":True}))
]

cur.executemany("INSERT INTO access_policy (row_key, value) VALUES (?, ?);", policy_data)

conn.commit()
print("✓ Đã insert dữ liệu thành công")

# Hàm hỗ trợ JSON extraction cho SQLite
def json_extract(data, key):
    try:
        data_dict = json.loads(data)
        return data_dict.get(key)
    except:
        return None

# df2 = pd.DataFrame(cur.fetchall(), columns=['user', 'file'])
# print(df2)

# # Đóng kết nối
# cur.close()
# conn.close()
# print("\n✓ Hoàn thành với SQLite in-memory")

Đang tạo các bảng...
✓ Đã tạo các bảng thành công

Đang insert dữ liệu...
✓ Đã insert dữ liệu thành công


In [50]:
# =========================================
# XEM TOÀN BỘ DỮ LIỆU - Chạy riêng script này
# =========================================

print("\n--- XEM TOÀN BỘ DỮ LIỆU TRONG CÁC BẢNG ---")

print("\n📋 Bảng SUBJECTS (Người dùng):")
cur.execute("SELECT row_key, value FROM subjects;")
subjects = cur.fetchall()
for row in subjects:
    print(f"{row[0]}: {json.loads(row[1])}")

print("\n📁 Bảng OBJECTS (Tài nguyên):")
cur.execute("SELECT row_key, value FROM objects;")
objects = cur.fetchall()
for row in objects:
    print(f"{row[0]}: {json.loads(row[1])}")

print("\n🔐 Bảng ACCESS_POLICY (Chính sách truy cập):")
cur.execute("SELECT row_key, value FROM access_policy;")
policies = cur.fetchall()
for row in policies:
    print(f"{row[0]}: {json.loads(row[1])}")


--- XEM TOÀN BỘ DỮ LIỆU TRONG CÁC BẢNG ---

📋 Bảng SUBJECTS (Người dùng):
user:1: {'username': 'alice', 'group_name': 'dev', 'department': 'it'}
user:2: {'username': 'bob', 'group_name': 'dev', 'department': 'it'}
user:3: {'username': 'charlie', 'group_name': 'qa', 'department': 'it'}
user:4: {'username': 'david', 'group_name': 'recruiting', 'department': 'hr'}
user:5: {'username': 'eve', 'group_name': 'payroll', 'department': 'hr'}
user:6: {'username': 'frank', 'group_name': 'north', 'department': 'sales'}
user:7: {'username': 'grace', 'group_name': 'south', 'department': 'sales'}

📁 Bảng OBJECTS (Tài nguyên):
file:1: {'company': 'ABC', 'branch': 'engineering', 'file_name': 'product_spec.pdf'}
file:2: {'company': 'ABC', 'branch': 'engineering', 'file_name': 'architecture.doc'}
file:3: {'company': 'ABC', 'branch': 'research', 'file_name': 'research_paper.pdf'}
file:4: {'company': 'financecorp', 'branch': 'main', 'file_name': 'financial_report.xlsx'}
file:5: {'company': 'financecorp', 

In [45]:
# =========================================
# 3.1 USER LEVEL - Chạy riêng script này
# =========================================

print("\n--- 3.1 USER LEVEL (alice truy cập file của ABC/engineering) ---")
print("Query: Tìm file mà alice được truy cập tại ABC/engineering")

cur.execute("""
SELECT 
    json_extract(value, '$.file_name') AS file,
    json_extract(value, '$.username') AS username,
    json_extract(value, '$.company') AS company,
    json_extract(value, '$.branch') AS branch
FROM access_policy
WHERE json_extract(value, '$.username') = 'alice'
  AND json_extract(value, '$.company') = 'ABC'
  AND json_extract(value, '$.branch') = 'engineering';
""")

results = cur.fetchall()
if results:
    df1 = pd.DataFrame(results, columns=['file', 'username', 'company', 'branch'])
    print("Kết quả:")
    print(df1)
    print(f"\n→ Alice được truy cập {len(results)} file tại ABC/engineering")
else:
    print("Không có kết quả phù hợp")


--- 3.1 USER LEVEL (alice truy cập file của ABC/engineering) ---
Query: Tìm file mà alice được truy cập tại ABC/engineering
Kết quả:
               file username company       branch
0  product_spec.pdf    alice     ABC  engineering
1  architecture.doc    alice     ABC  engineering

→ Alice được truy cập 2 file tại ABC/engineering


In [46]:
# =========================================
# 3.2 GROUP LEVEL - Chạy riêng script này
# =========================================

print("\n--- 3.2 GROUP LEVEL (nhóm dev/it truy cập ABC/engineering) ---")
print("Query: Tìm user và file mà nhóm dev/it được truy cập tại ABC/engineering")

cur.execute("""
SELECT 
    json_extract(value, '$.username') AS user,
    json_extract(value, '$.file_name') AS file,
    json_extract(value, '$.group_name') AS group_name,
    json_extract(value, '$.department') AS department
FROM access_policy
WHERE json_extract(value, '$.group_name') = 'dev'
  AND json_extract(value, '$.department') = 'it'
  AND json_extract(value, '$.company') = 'ABC'
  AND json_extract(value, '$.branch') = 'engineering';
""")

results = cur.fetchall()
if results:
    df2 = pd.DataFrame(results, columns=['user', 'file', 'group_name', 'department'])
    print("Kết quả:")
    print(df2)
    print(f"\n→ Nhóm dev/it được truy cập {len(results)} file tại ABC/engineering")
else:
    print("Không có kết quả phù hợp")


--- 3.2 GROUP LEVEL (nhóm dev/it truy cập ABC/engineering) ---
Query: Tìm user và file mà nhóm dev/it được truy cập tại ABC/engineering
Kết quả:
  user              file group_name department
0  bob  product_spec.pdf        dev         it

→ Nhóm dev/it được truy cập 1 file tại ABC/engineering


In [47]:
# =========================================
# 3.3 DEPARTMENT LEVEL - Chạy riêng script này
# =========================================

print("\n--- 3.3 DEPARTMENT LEVEL (phòng hr truy cập ABC/research) ---")
print("Query: Tìm user và file mà phòng HR được truy cập tại ABC/research")

cur.execute("""
SELECT 
    json_extract(value, '$.username') AS user,
    json_extract(value, '$.file_name') AS file,
    json_extract(value, '$.department') AS department,
    json_extract(value, '$.company') AS company,
    json_extract(value, '$.branch') AS branch
FROM access_policy
WHERE json_extract(value, '$.department') = 'hr'
  AND json_extract(value, '$.company') = 'ABC'
  AND json_extract(value, '$.branch') = 'research';
""")

results = cur.fetchall()
if results:
    df3 = pd.DataFrame(results, columns=['user', 'file', 'department', 'company', 'branch'])
    print("Kết quả:")
    print(df3)
    print(f"\n→ Phòng HR được truy cập {len(results)} file tại ABC/research")
else:
    print("Không có kết quả phù hợp")


--- 3.3 DEPARTMENT LEVEL (phòng hr truy cập ABC/research) ---
Query: Tìm user và file mà phòng HR được truy cập tại ABC/research
Không có kết quả phù hợp


In [48]:
# =========================================
# 3.4 BRANCH LEVEL - Chạy riêng script này
# =========================================

print("\n--- 3.4 BRANCH LEVEL (tất cả user truy cập ABC/engineering) ---")
print("Query: Tìm tất cả user và file được truy cập tại ABC/engineering")

cur.execute("""
SELECT 
    json_extract(value, '$.username') AS user,
    json_extract(value, '$.file_name') AS file,
    json_extract(value, '$.company') AS company,
    json_extract(value, '$.branch') AS branch
FROM access_policy
WHERE json_extract(value, '$.company') = 'ABC'
  AND json_extract(value, '$.branch') = 'engineering';
""")

results = cur.fetchall()
if results:
    df4 = pd.DataFrame(results, columns=['user', 'file', 'company', 'branch'])
    print("Kết quả:")
    print(df4)
    print(f"\n→ Có {len(results)} quyền truy cập tại ABC/engineering")
else:
    print("Không có kết quả phù hợp")


--- 3.4 BRANCH LEVEL (tất cả user truy cập ABC/engineering) ---
Query: Tìm tất cả user và file được truy cập tại ABC/engineering
Kết quả:
    user              file company       branch
0  alice  product_spec.pdf     ABC  engineering
1  alice  architecture.doc     ABC  engineering
2    bob  product_spec.pdf     ABC  engineering

→ Có 3 quyền truy cập tại ABC/engineering



--- XEM TOÀN BỘ DỮ LIỆU TRONG CÁC BẢNG ---

📋 Bảng SUBJECTS (Người dùng):
user:1: {'username': 'alice', 'group_name': 'dev', 'department': 'it'}
user:2: {'username': 'bob', 'group_name': 'dev', 'department': 'it'}
user:3: {'username': 'charlie', 'group_name': 'qa', 'department': 'it'}
user:4: {'username': 'david', 'group_name': 'recruiting', 'department': 'hr'}
user:5: {'username': 'eve', 'group_name': 'payroll', 'department': 'hr'}
user:6: {'username': 'frank', 'group_name': 'north', 'department': 'sales'}
user:7: {'username': 'grace', 'group_name': 'south', 'department': 'sales'}

📁 Bảng OBJECTS (Tài nguyên):
file:1: {'company': 'ABC', 'branch': 'engineering', 'file_name': 'product_spec.pdf'}
file:2: {'company': 'ABC', 'branch': 'engineering', 'file_name': 'architecture.doc'}
file:3: {'company': 'ABC', 'branch': 'research', 'file_name': 'research_paper.pdf'}
file:4: {'company': 'financecorp', 'branch': 'main', 'file_name': 'financial_report.xlsx'}
file:5: {'company': 'financecorp', 